# Aubert EMAKO

# Deep learning challenge using Bidirectional LSTM model

# IMPORT THE DATA FROM KAGGLE

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle


In [0]:
#Upload the kaggle.json file that contains the key for the Kaggle API
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c efreiparisdeeplearning2019

  0% 0.00/315k [00:00<?, ?B/s]
100% 315k/315k [00:00<00:00, 43.6MB/s]
  0% 0.00/1.34M [00:00<?, ?B/s]
100% 1.34M/1.34M [00:00<00:00, 88.4MB/s]
100% 27.4M/27.4M [00:00<00:00, 38.3MB/s]



In [0]:
ls

dataset_test_no_labels.csv.zip  example_submission_test.csv  sample_data/
dataset_train.csv.zip           kaggle.json


# IMPORT THE LIBRARIES

In [0]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax
from numpy import asarray
from numpy import zeros

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import urllib.request
import zipfile
import os

import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder

from numpy import array

import nltk
from nltk.corpus import stopwords 
import string
import keras
from keras.utils import to_categorical
from keras import regularizers
from keras import utils as np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras.models import Model, Sequential, model_from_json, load_model
from keras.layers import Input, Embedding, SpatialDropout1D, LSTM, Dense, Dropout, Activation, Flatten, Conv1D, GlobalMaxPooling1D, Concatenate, Bidirectional
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import concatenate
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# PREPARE THE DATA

In [0]:
# Import the test and train datasets into pandas dataframe
df_train = pd.read_csv('dataset_train.csv.zip', compression='zip', sep='\t')
df_test = pd.read_csv('dataset_test_no_labels.csv.zip', compression='zip', sep='\t')

#df_train.groupby(['label'])['label'].count()
# --> The data is balanced

# Shuffle the data  
df_train = sklearn.utils.shuffle(df_train, random_state=42)


# Remove stopwords
# stop_words = set(stopwords.words('english'))

# def clean_column(col):
#   col = col.str.lower()
#   col = col.str.replace('[^\w\s]','  ')
#   col = col.apply(lambda x: '  '.join([word for word in x.split() if word not in (stop_words)]))

#   return col

# df_train.sentence_1 = clean_column(df_train.sentence_1)
# df_train.sentence_2 = clean_column(df_train.sentence_2)


# Encode the labels because the model requires numbers, not text
label_encoder = LabelEncoder()
df_train.label = label_encoder.fit_transform(df_train.label)

X = df_train[['sentence_1', 'sentence_2']]
y = df_train['label']

# Splitting the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Keras models require one hot encoding for the label input
y_train = pd.get_dummies(y_train)
#y_test = pd.get_dummies(y_test)

# Vectorize the text corpus
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train.sentence_1+X_train.sentence_2)
vocab_size = len(word_tokenizer.word_index) + 1
# vocab_size = 10000
max_length = 100

print('vocabulary size: ', vocab_size)
print('max length: ', max_length)

# Text to sequence, then pad the sequence for the train set and the test set
# TRAIN
train_encoded_sentences = word_tokenizer.texts_to_sequences(X_train['sentence_1'])
train_encoded_sentences2 = word_tokenizer.texts_to_sequences(X_train['sentence_2'])
padded_sentences = pad_sequences(train_encoded_sentences, max_length, padding='post')
padded_sentences2 = pad_sequences(train_encoded_sentences2, max_length, padding='post')

# TEST
# test_encoded_sentences = word_tokenizer.texts_to_sequences(X_test['sentence_1'])
# test_encoded_sentences2 = word_tokenizer.texts_to_sequences(X_test['sentence_2'])
# padded_sentences_test = pad_sequences(test_encoded_sentences, max_length, padding='post')
# padded_sentences2_test = pad_sequences(test_encoded_sentences2, max_length, padding='post')

print('X train shape: ', X_train.shape)
print('y train shape: ', y_train.shape)
# print('X test shape: ', X_test.shape)
# print('y test shape: ', y_test.shape)

print('word count shape: ', len(word_tokenizer.word_counts))

nombre de mots:  111909
X train:  (392662, 2)
y train:  (392662, 3)
word count shape:  111908


# GLOVE

In [0]:
# Let's use a pretrained glove embedding
glove_zip_file = "glove.6B.zip"
glove_vectors_file = "glove.6B.50d.txt"

if (not os.path.isfile(glove_zip_file) and
    not os.path.isfile(glove_vectors_file)):
    urllib.request.urlretrieve ("http://nlp.stanford.edu/data/glove.6B.zip", 
                 glove_zip_file)
    

def unzip_single_file(zip_file_name, output_file_name):
# If the outFile is already created, don't recreate
# If the outFile does not exist, create it from the zipFile
    if not os.path.isfile(output_file_name):
        with open(output_file_name, 'wb') as out_file:
            with zipfile.ZipFile(zip_file_name) as zipped:
                for info in zipped.infolist():
                    if output_file_name in info.filename:
                        with zipped.open(info) as requested_file:
                            out_file.write(requested_file.read())
                            return

unzip_single_file(glove_zip_file, glove_vectors_file)

embeddings_dictionary = dict()

glove_wordmap = {}
with open(glove_vectors_file, "r") as glove:
    for line in glove:
        records = line.split()
        word = records[0]
        vector_dimensions = asarray(records[1:], dtype='float32')
        embeddings_dictionary [word] = vector_dimensions
    glove.close()

embedding_matrix = zeros((vocab_size, 50))
for word, index in word_tokenizer.word_index.items():
  embedding_vector = embeddings_dictionary.get(word)
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector


print('embedding dictionary shape: ', len(embeddings_dictionary))
print('embedding matrix shape: ', embedding_matrix.shape)

embedding dictionary shape:  400000
embedding matrix shape:  (111909, 50)


# MODEL 

In [0]:
#Applying Embedding and Bidirectional LSTM to both sentences separately, then using dropout and flattening
input1 = Input(shape=(padded_sentences.shape[1],))
x1 = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=max_length, trainable=False)(input1)
x1 = Bidirectional(LSTM(64, return_sequences=True))(x1)
x1 = Dropout(0.3)(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(padded_sentences2.shape[1],))
x2 = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=max_length, trainable=False)(input2)
x2 = Bidirectional(LSTM(64, return_sequences=True))(x2)
x2 = Dropout(0.3)(x2)
x2 = Flatten()(x2)

# Concatenates the two outputs
out = concatenate([x1, x2])

#interpretation

out = Dense(64, activation='relu')(out)
out = Dense(3, activation='softmax')(out) 

model = Model(inputs=[input1, input2], outputs=out)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 1024

model.fit([padded_sentences, padded_sentences2], y_train, batch_size = batch_size, epochs = 10, validation_split=0.15)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 50)      5595450     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 50)      5595450     i

In [0]:
#To see the results on the testing part of the dataset if the data is split when preparing the data
# score = model.evaluate([padded_sentences_test, padded_sentences2_test], y_test)
# print('Test data loss:', score[0])
# print('Test data accuracy:', score[1])

# Prediction of the labels for the Test Dataset

In [0]:
test_X = df_test[['sentence_1', 'sentence_2']]

sumbit_encoded_sentences = word_tokenizer.texts_to_sequences(test_X['sentence_1'])
sumbit_encoded_sentences2 = word_tokenizer.texts_to_sequences(test_X['sentence_2'])

submit_padded = pad_sequences(sumbit_encoded_sentences, max_length, padding='post')
submit_padded2 = pad_sequences(sumbit_encoded_sentences2, max_length, padding='post')

predicted_labels = model.predict([submit_padded, submit_padded2]).argmax(axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels)
print(predicted_labels)

my_submission = pd.DataFrame({'index': test_X.index, 'label': predicted_labels})
my_submission.to_csv('submission.csv', index=False)
print(my_submission.shape)
my_submission.head()


['entailment' 'neutral' 'entailment' ... 'contradiction' 'neutral'
 'neutral']
(19647, 2)


,index,label
0,0,entailment
1,1,neutral
2,2,entailment
3,3,entailment
4,4,contradiction


In [0]:
#Download the file for submission
files.download('submission.csv')